In [1]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import *
from tensorflow.keras.applications import ResNet50

2025-12-13 02:27:47.663707: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [12]:
train_dir = "/home/kanishka/Desktop/Fruits-360/data/fruits-360_100x100/fruits-360/Training"
test_dir = "/home/kanishka/Desktop/Fruits-360/data/fruits-360_100x100/fruits-360/Test"

In [13]:
train_tfms = tf.keras.Sequential([
    tf.keras.layers.Resizing(224, 224),
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Normalization(
        mean=[0.485, 0.456, 0.406],
        variance=[0.229**2, 0.224**2, 0.225**2]
    )
])

In [20]:
base_model = ResNet50(
    weights=None,
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.load_weights('./model/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')

base_model.trainable = False

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = './model/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32
)
num_classes = len(train_ds.class_names)
print(f"Number of classes: {num_classes}")

In [15]:
model = tf.keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

NameError: name 'base_model' is not defined

In [ ]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=32
)

train_ds = train_ds.map(lambda x, y: (train_tfms(x, training=True), y))

history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10
)